In [1]:
import numpy as np
import pandas as pd

import os
import re
import ssl

import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# create a ssl unverfified context
ssl._create_default_https_context = ssl._create_unverified_context


In [31]:
dataset_dir = '../../data/stack_overflow'
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')
print('train_dir', os.listdir(train_dir))
print('test_dir', os.listdir(test_dir))

train_dir ['python', 'java', 'csharp', 'javascript']
test_dir ['python', 'java', 'csharp', 'javascript']


In [32]:
batch_size=32
seed=42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [33]:
raw_train_ds

<BatchDataset shapes: ((None,), (None,)), types: (tf.string, tf.int32)>

In [34]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(2):
        print('Question', text_batch.numpy()[i])
        print('Label', label_batch.numpy()[i])

Question b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can be easily fixed, please forgive me. my program has a tester class with a main. when i send that to my regularpolygon class, it sends it to the wrong constructor. i have two constructors. 1 without perameters..public regularpolygon().    {.       mynumsides = 5;.       mysidelength = 30;.    }//end default constructor...and my second, with perameters. ..public regularpolygon(int numsides, double sidelength).    {.        mynumsides = numsides;.        mysidelength = sidelength;.    }// end constructor...in my tester class i have these two lines:..regularpolygon shape = new regularpolygon(numsides, sidelength);.        shape.menu();...numsides and sidelength were declared and initialized earlier in the testing class...so what i want to happen, is the tester class sends numsides and sidelength to the second constructor and use it in that class. but it only uses the default const

In [35]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [36]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size
)

Found 8000 files belonging to 4 classes.


In [37]:
for class_name in raw_train_ds.class_names:
    print(class_name)

csharp
java
javascript
python


In [38]:
# data preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)

In [39]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [40]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [41]:
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print('Question', first_question)
print('Label', raw_train_ds.class_names[first_label])
print('Vectorized Question', vectorize_text(first_question, first_label))

Question tf.Tensor(b'"unit testing of setters and getters teacher wanted us to do a comprehensive unit test. for me, this will be the first time that i use junit. i am confused about testing set and get methods. do you think should i test them? if the answer is yes; is this code enough for testing?..  public void testsetandget(){.    int a = 10;.    class firstclass = new class();.    firstclass.setvalue(10);.    int value = firstclass.getvalue();.    assert.asserttrue(""error"", value==a);.  }...in my code, i think if there is an error, we can\'t know that the error is deriving because of setter or getter."\n', shape=(), dtype=string)
Label java
Vectorized Question (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[1011,  773,    9, 2456,    8, 1863, 2362,  690, 1267,    4,   40,
           5,    1, 1011,  196,   12,   74,   13,   72,   33,    2,   98,
         105,   14,    3,   70, 9611,    3,   34,  888,  202,  773,  107,
           8,   41,  242,   40,   58,  291,   90,    3

In [43]:
print('1011 -->', vectorize_layer.get_vocabulary()[1011])
print(' 773 -->', vectorize_layer.get_vocabulary()[773])
print('Vocabulary size', len(vectorize_layer.get_vocabulary()))


1011 --> unit
 773 --> testing
Vocabulary size 10000


In [45]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [47]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [49]:
embedding_dim = 16

model = tf.keras.Sequential([
    layers.Embedding(max_features + 1, embedding_dim),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(4)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160016    
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 68        
Total params: 160,084
Trainable params: 160,084
Non-trainable params: 0
_________________________________________________________________


In [50]:
model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

In [51]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
200/200 [==============================] - 2s 10ms/step - loss: 1.3804 - accuracy: 0.3416 - val_loss: 1.3703 - val_accuracy: 0.4306
Epoch 2/10
200/200 [==============================] - 1s 3ms/step - loss: 1.3531 - accuracy: 0.4734 - val_loss: 1.3335 - val_accuracy: 0.5631
Epoch 3/10
200/200 [==============================] - 1s 3ms/step - loss: 1.3053 - accuracy: 0.5509 - val_loss: 1.2761 - val_accuracy: 0.6000
Epoch 4/10
200/200 [==============================] - 1s 3ms/step - loss: 1.2378 - accuracy: 0.5958 - val_loss: 1.2066 - val_accuracy: 0.6219
Epoch 5/10
200/200 [==============================] - 1s 3ms/step - loss: 1.1629 - accuracy: 0.6369 - val_loss: 1.1328 - val_accuracy: 0.6712
Epoch 6/10
200/200 [==============================] - 1s 3ms/step - loss: 1.0869 - accuracy: 0.6705 - val_loss: 1.0629 - val_accuracy: 0.6925
Epoch 7/10
200/200 [==============================] - 1s 3ms/step - loss: 1.0174 - accuracy: 0.7017 - val_loss: 0.9987 - val_accuracy: 0.7244
Epoch

In [52]:
loss, accuracy = model.evaluate(test_ds)
print('Loss', loss)
print('Accuracy', accuracy)

250/250 [==============================] - 3s 11ms/step - loss: 0.8749 - accuracy: 0.7321
Loss 0.8748984336853027
Accuracy 0.7321249842643738
